In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7,8,9"

import faiss
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 路径设置
tokenizer_path = "../../BAAI_bge-m3"
gen_model_path = "../../GLM-4-9B-Chat"

# 加载tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModel.from_pretrained(tokenizer_path)

# 加载生成模型和tokenizer
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_path, trust_remote_code=True)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_path, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, trust_remote_code=True)

# 设置设备为cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
gen_model = gen_model.to(device).eval()

# 固定随机种子
torch.manual_seed(42)

# 加载FAISS索引
index_path = "../faiss_index/embedding.index"
index = faiss.read_index(index_path)

# 加载条目和文件名映射
entries = []
with open("../faiss_index/entries.txt", "r", encoding="utf-8") as f:
    for line in f:
        file_path, entry = line.strip().split('\t')
        entries.append((file_path, entry))

# 函数：计算余弦相似度
def cosine_similarity_embeddings(embeddings):
    return cosine_similarity(embeddings)

# 函数：进行检索并去重
def search(query, top_k=5):
    # 对查询进行编码
    query_tokens = tokenizer(query, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length)["input_ids"].to(device)
    with torch.no_grad():
        query_embedding = model(query_tokens).last_hidden_state.mean(dim=1).cpu().numpy()

    # 检索最相似的top_k个结果
    distances, indices = index.search(query_embedding, top_k)
    results = [(entries[I], distances[0][j]) for j, I in enumerate(indices[0])]
    
    # 获取条目的嵌入向量
    entry_embeddings = []
    for (file_path, entry), distance in results:
        entry = str(entry)  # 确保entry是字符串
        entry_tokens = tokenizer(entry, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length)["input_ids"].to(device)
        with torch.no_grad():
            entry_embedding = model(entry_tokens).last_hidden_state.mean(dim=1).cpu().numpy()
        entry_embeddings.append(entry_embedding)

    # 计算条目之间的相似度并去重
    entry_embeddings = np.vstack(entry_embeddings)
    similarity_matrix = cosine_similarity_embeddings(entry_embeddings)
    
    unique_results = []
    seen_indices = set()
    
    for i in range(len(results)):
        if i in seen_indices:
            continue
        similar_indices = [j for j in range(len(results)) if similarity_matrix[i][j] > 0.8 and j != i]
        similar_indices.append(i)
        longest_entry = max(similar_indices, key=lambda x: len(results[x][0][1]))
        unique_results.append(results[longest_entry])
        seen_indices.update(similar_indices)

    return unique_results, query_embedding

# 函数：生成答案
def generate_answer(context, query):
    input_text = f"法律问题:{query}\n回答可能会用到的参考文献:{context}\n"
    inputs = gen_tokenizer(input_text, return_tensors="pt", truncation=True, max_length=gen_tokenizer.model_max_length).to(device)
    gen_kwargs = {"max_length": 1024, "do_sample": True}  # 禁用采样

    with torch.no_grad():
        outputs = gen_model.generate(**inputs, **gen_kwargs)
        answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 去除input_text相关内容
    answer = answer.replace(input_text, "").strip()
    return answer

# 函数：进行检索并生成答案
def search_and_generate(query, top_k=5):
    results, query_embedding = search(query, top_k)
    context = "\n".join([entry for (file_path, entry), distance in results])
    response = generate_answer(context, query)
    return response, results

# 示例查询
query = input("请输入您的法律问题：")
answer, results = search_and_generate(query, top_k=10)  # top_k 设为10以获取更多候选结果用于去重

print(f"基于参考文献的回答: {answer}")
print("参考文献:")
for (filename, entry), distance in results:
    print(f"文件: {filename}, 条目: {entry.strip()}, 距离: {distance}")

/home/yuwenhan/anaconda3/envs/GLM_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 10/10 [00:01<00:00,  7.65it/s]


基于参考文献的回答: 如果您在驾驶车辆时不小心撞人了，这是一个严重的交通事故，以下是一些建议的步骤：

1. **立即停车**：首先，请立即停车确保安全，不要离开现场。

2. **报警处理**：立即拨打110报警，并告知警方事故发生的具体情况。

3. **救治伤者**：如果可能的话，立即对伤者进行初步救助。如果伤者受伤严重，应立即拨打120急救电话。

4. **保护现场**：如果现场没有严重危险，应尽量保持现场原状，等待交警到来。

5. **等待交警到来**：交警到达现场后，他们会进行调查，并制作事故认定书。

6. **通知保险公司**：立即通知您的保险公司，并按照保险公司的指导处理相关事宜。

7. **承担责任**：
   - **过错责任**：根据《道路交通安全法》第76条的规定，如果交警认定您在此事故中有过错，您需要承担相应的赔偿责任。
   - **无过错责任**：虽然通常机动车与非机动车驾驶人、行人之间发生的交通事故责任适用无过错责任原则，但如果有证据证明非机动车驾驶人、行人有过错，根据过错程度，可能会适当减轻您的赔偿责任。

8. **后续追责**：
   - 如果您认为事故责任应由对方承担，可以进一步收集证据并向法院提起民事诉讼。
   - 如果需要法律援助，可以联系法律服务机构。

9. **心理调适**：交通事故对您可能带来心理压力，可以考虑寻求专业心理辅导。

最后，为了避免这类事件的发生，建议您平时遵守交通规则，提高驾驶安全意识。

（以上内容仅供参考，具体情况还需依据实际情况和相关法律法规进行判断。）参考文献：《道路交通安全法》第76条。机动车交通事故责任、机动车交通事故责任的归责原则等内容参考自《道路交通安全法》。如果您需要更详细的法律咨询，建议您咨询专业法律人士。
参考文献:
文件: ../reference_book/民法/第三十二章特殊侵权责任.txt, 条目: "机动车交通事故责任 机动车交通事故责任的归责原则 机动车交通事故责任的归责原则是过错责任和无过错责任的结合。一方面，机动车之间发生的交通事故责任适用过错责任原则；另一方面，机动车与非机动车驾驶人.行人之间发生的交通事故责任适用无过错责任原则。机动车交通事故责任的归责原则在我国体现于《道路交通安全法》第76条的规定，即机动车发生交通事故造成人身伤亡.财产损失的，由

In [2]:
print(results)

[(('../reference_book/民法/第三十二章特殊侵权责任.txt', '  "机动车交通事故责任 机动车交通事故责任的归责原则 机动车交通事故责任的归责原则是过错责任和无过错责任的结合。一方面，机动车之间发生的交通事故责任适用过错责任原则；另一方面，机动车与非机动车驾驶人.行人之间发生的交通事故责任适用无过错责任原则。机动车交通事故责任的归责原则在我国体现于《道路交通安全法》第76条的规定，即机动车发生交通事故造成人身伤亡.财产损失的，由保险公司在机动车第三者责任强制保险责任限额范围内予以赔偿；不足的部分，按照下列规定承担赔偿责任:(1)机动车之间发生交通事故的，由有过错的一方承担赔偿责任；双方都有过错的，按照各自过错的比例分担责任。(2)机动车与非机动车驾驶人.行人之间发生交通事故，非机动车驾驶人.行人没有过错的，由机动车一方承担赔偿责任；有证据证明非机动车驾驶人.行人有过错的，根据过错程度适当减轻机动车一方的赔偿责任；机动车一方没有过错的，承担不超过10%的赔偿责任。交通事故的损失是由非机动车驾驶人.行人故意碰撞机动车造成的，机动车一方不承担赔偿责任。",'), 276.5871)]


In [7]:
# 函数：进行检索并去重
def search(query, top_k=5):
    # 对查询进行编码
    query_tokens = tokenizer(query, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length)["input_ids"].to(device)
    with torch.no_grad():
        query_embedding = model(query_tokens).last_hidden_state.mean(dim=1).cpu().numpy()

    # 检索最相似的top_k个结果
    distances, indices = index.search(query_embedding, top_k)
    results = [(entries[I], distances[0][j]) for j, I in enumerate(indices[0])]
    
    # 获取条目的嵌入向量
    entry_embeddings = []
    for (file_path, entry), distance in results:
        entry = str(entry)  # 确保entry是字符串
        entry_tokens = tokenizer(entry, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length)["input_ids"].to(device)
        with torch.no_grad():
            entry_embedding = model(entry_tokens).last_hidden_state.mean(dim=1).cpu().numpy()
        entry_embeddings.append(entry_embedding)

    # 计算条目之间的相似度并去重
    entry_embeddings = np.vstack(entry_embeddings)
    similarity_matrix = cosine_similarity_embeddings(entry_embeddings)
    
    unique_results = []
    seen_indices = set()
    
    for I in range(len(results)):
        if I in seen_indices:
            continue
        similar_indices = [j for j in range(len(results)) if similarity_matrix[I][j] > 0.8 and j != I]
        similar_indices.append(I)
        longest_entry = max(similar_indices, key=lambda x: len(results[x][0][1]))
        unique_results.append(results[longest_entry])
        seen_indices.update(similar_indices)

    return unique_results, query_embedding
    

In [8]:
query1 = "我国的立法依据是什么？"
search(query1)


([(('../reference_book/法理学/第二章  法的运行.txt',
    '  "立法 立法体制 根据宪法和立法法的有关规定，我国国家机关的立法权限划分如下: 多级并存，即全国人大及其常委会制定国家法律，国务院及其所属部委分别制定行政法规和部门规章，地方权力机关及其人民政府制定地方性法规和地方政府规章。这些不同主体制定的规范性法律文件之间存在效力上的高低之分，低层次的规范性法律文件不得同高层次的规范性法律文件相抵触。",'),
   216.72443)],
 array([[-0.4623774 ,  0.21046692, -0.10280899, ..., -0.8041831 ,
         -0.85859144, -0.31263992]], dtype=float32))